## Closer look at lists

- We have explored a few of the built-in methods of a `List` in Scala
  ```scala
    type List[Fruit]
    val fruits = List("apple", "orange", "banana")
    val nums = 1 :: 2 :: Nil //Nil is built in

    nums match
      case x :: y:: _ => x+y //3
      case _ => nums
    
    fruits.isEmpty //false
    fruits.length //3

    fruits.head //"apple"
    fruits.last //"banana"

    fruits.init //List("apple", "orange")
    fruits.tail //List("orange", "banana")

    fruits.take(2) //List("apple", "orange")
    fruits.drop(2) //List("banana")
    fruits(2) //"banana"

    fruits ++ List("pineapple") //List("apple", "orange", "banana", "pineapple")
    fruits.reverse //List("banana", "orange", "apple")
    fruits.updated(0, "mango") //List("mango", "orange", "banana")

    fruits.indexOf("apple") //0
    fruits.contains("watermelon") //false

  ```

### Implementation of some list methods

- To de-mystify some of the methods above, we'll implement some of them from scratch. Let `val l: List[Any]`. We will implement:
  - `last(l)`
  - `init(l)`
  - `l ++ m`
  - `l.reverse`
  - `removeAt(n: Int, l)`
  - `flatten(l)`

- See `1-slides.scala`

## Tuples and Generic Methods

- Tuples in Scala are similar to list, with the following differences
  - They can be heterogeneous
  - They cannot be extended
  - Element access can be done via `t._1`, `t._2`, ...

- To see how this can be applied (indirectly), we'll write a function to do merge sort

- Note the implementation accounts for `mergeSort` that takes in any parameter type T, and so lets you pass in an arbitrary comparison function `lt`

- See `2-slides.scala`

## Higher-Order List Functions

- In Scala, functions can be combined to give you higher order functions

- We have already seen some repeated patterns being used in Scala; transforming elements in a list, retrieving elements that meet a criteria, combine elements with some operator

- For example, let's suppose we want to write a function that takes a list, and multiples all elements by 2
  ```scala
    def multByFactor(l: List[Int], factor: Int): List[Int] = {
      l match
        case Nil => Nil
        case elem :: tail => elem*factor :: multByFactor(l, factor)
    }

    multByFactor(l, 2)
  ```

- We could, however, just as easily write a general function that follows this pattern of applying a function to every element in a list!

  ```scala
    extension [T](l: List[T])
      def map[U](f: T => U): List[U] = {
        l match
          case Nil => Nil
          case elem :: tail => f(elem) :: tail.map(f)
      }

    l.map(x => 2*x)
  ```

- The idea here is to go one step beyond functions; we write functions that allow us to apply arbitrary function patterns

### Exercise 1

- Consider a function to square each element of a list, and return the result. Complete the two following equivalent definitions of squareList.

  ```scala
    def squareList(xs: List[Int]): List[Int] = xs match
      case Nil => Nil
      case y :: ys => y*y :: squareList(ys)
    
    def squareList(xs: List[Int]): List[Int] =
      xs.map(x => x*x)
  ```



### Higher Level Function: Filtering

- Same idea as above; but generalising for `filter`
  ```scala
    def posElems(xs: List[Int]): List[Int] = xs match
      case Nil => xs
      case y :: ys => if y > 0 then y :: posElems(ys) else posElems(ys)

    extension [T](l: List[T])
      def filter(f: T => Boolean): List[T] = {
        l match
          case Nil => Nil
          case elem :: tail => if f(elem) then elem :: tail.filter(f) else tail.filter(f)
      }

    l.filter(x => x > 0) //equivalent
    posElems(l) //equivalent
  ```

### Exercise 2

- Let's try to apply this concept for the following functions
  - `l.filterNot(f)`: filter elements that do not meet condition `f`
  - `l.partition(f)`: returns a tuple of Lists, with one where all elements meet condition `f`, and one where all elements do not
  - `l.takeWhile(f)`: starting from l.head, while f is true, append element to result, and stop when f is false
  - `l.dropWhile(f)`: starting from l.head, while f is true, drop element from list, and stop when f is false
  - `l.span(f)`: returns a tuple of lists, with first element being `l.takeWhile(f)` and second being `l.dropWhile(f)`

- See `3-slides.scala`

### Exercise 3

- Write a function pack that packs consecutive duplicates of list elements into sublists. For instance, `pack(List("a", "a", "a", "b", "c", "c", "a"))` should give `List(List("a", "a", "a"), List("b"), List("c", "c"), List("a"))`

- See `3-slides.scala`

### Exercise 4

- Using `pack`, write a function encode that produces the run-length encoding of a list. The idea is to encode n consecutive duplicates of an element x as a pair `(x, n)`. For instance, `encode(List("a", "a", "a", "b", "c", "c", "a"))` should give `List(("a", 3), ("b", 1), ("c", 2), ("a", 1))`

- See `3-slides.scala`

## Reduction of Lists

- Scala's design makes it deliberately hard to do any type of iteration of objects. It is more idiomatic to things like `reduce`
  - For instance, if you want to do something like `sum([1,2,3])`, you will need to write your own iterator

- There are 2 main iterators in Scala to be familiar with; `foldLeft` and `foldRight`
  - Technically there is also `reduceLeft` and `reduceRight`
  - However, `reduce*` doesn't give you the ability to specify a base case, which makes it a lot less flexible. So conceptually, all you really need is `fold*`

- To illustrate the use of `foldLeft` and `foldRight`, let's imagine that we wish to concatenate 2 Lists together. Notwithstanding the fact that there is an obvious syntax to do this (`++` or `:::`), let's try to implement it via `foldLeft` and `foldRight`

  ```scala
  val l1 = List(1,2,3)
  val l2 = List(10,20,30)

  /*
    Accumulates into l2 by appending elements of l1 into l2 from left to right
    ((List(10,20,30) :+ 1) :+ 2) :+ 3
  */
  l1.foldLeft(l2)(_ :+ _) 

  /*
    Accumulates into l2 by prepending elements of l1 into l2 from right to left
    1 :: (2 :: (3 :: List(10,20,30)))
  */
  l1.foldRight(l2)(_ :: _) 
  ```

### Implementations of fold* and reduce*

- See `4-slides.scala`

### Exercise 1

- In this formulation of concat, it isn’t possible to replace foldRight by foldLeft. Why?

```scala
  def concat[T](xs: List[T], ys: List[T]): List[T] =
    xs.foldRight(ys)(_ :: _)
```

- Clearly, the issue is one of `type`
  - Recall that in `a.foldLeft(b)(f)`, we are doing `f(f(b,a(0)), a(1)...`
  - But in `a.foldRight(b)(f)`, we are doing `...f(a(N-1), f(a(N), b))`
  - In the example above, with fold right, you are doing `...(xs(N-1) :: (xs(N) :: ys))`, which makes sense because `::` expects the LHS to be a single value, and the RHs to be a list
  - If you change it to `foldLeft` however, this becomes `(ys :: xs(0)...` which is obviously wrong syntax for `::`

### Reversing List

- Previously, we implemented reverse in this way, which is an $O(N^2)$ implementation
  ```scala
  def MYreverse: List[T] = {
    l match 
      case Nil => Nil
      case elem :: tail => tail.MYreverse `MY++` List(elem)
  }
  ```

- But we can do better with this `foldLeft` syntax!!

- See `4-slides.scala`

### Exercise 2

- Complete the following definitions of the basic functions map and length on lists, such that their implementation uses foldRight:
  ```scala
    def mapFun[T, U](xs: List[T], f: T => U): List[U] =
      xs.foldRight(List[U]())( ??? )
      
    def lengthFun[T](xs: List[T]): Int =
      xs.foldRight(0)( ??? )
  ```

-  See `4-slides.scala`

## List Reasoning

- This is some proof by induction nonsense that doesn't seem super relevant IMO. Just read for fun